In [4]:
import mdtraj as md
import numpy as np

# Load the trajectory or structure file
# Replace 'structure_file.pdb' with the actual path to your PDB or CIF file
traj = md.load('fold_6zsl_model_0.cif')

# Inspect the topology to understand the structure
topology = traj.topology
print(topology)

<mdtraj.Topology with 3 chains, 612 residues, 4899 atoms, 4988 bonds>


In [5]:
# Select phosphate atoms (P) in RNA based on residue names (A, U, C, G)
rna_residue_names = ['A', 'U', 'C', 'G']
phosphate_indices = [atom.index for atom in topology.atoms if atom.name == 'P' and atom.residue.name in rna_residue_names]

# Select oxygen atoms in uracil bases in RNA
uracil_oxygen_indices = [atom.index for atom in topology.atoms if atom.element.symbol == 'O' and atom.residue.name == 'U']

# Check the selection
print(f"Phosphate atom indices: {phosphate_indices}")
print(f"Uracil oxygen atom indices: {uracil_oxygen_indices}")

Phosphate atom indices: [4739, 4759, 4779, 4799, 4819, 4839, 4859, 4879]
Uracil oxygen atom indices: [4738, 4740, 4741, 4742, 4745, 4747, 4749, 4753, 4756, 4760, 4761, 4762, 4765, 4767, 4769, 4773, 4776, 4780, 4781, 4782, 4785, 4787, 4789, 4793, 4796, 4800, 4801, 4802, 4805, 4807, 4809, 4813, 4816, 4820, 4821, 4822, 4825, 4827, 4829, 4833, 4836, 4840, 4841, 4842, 4845, 4847, 4849, 4853, 4856, 4860, 4861, 4862, 4865, 4867, 4869, 4873, 4876, 4880, 4881, 4882, 4885, 4887, 4889, 4893, 4896]


In [9]:
# Calculate the contacts without specifying a cutoff, then filter manually
# Pairwise contacts are defined as (atom.index, target_atom_index)
contact_pairs_phosphate = [(atom.index, phosphate) for atom in topology.atoms if atom.residue.is_protein for phosphate in phosphate_indices]
contact_pairs_uracil = [(atom.index, uracil_oxygen) for atom in topology.atoms if atom.residue.is_protein for uracil_oxygen in uracil_oxygen_indices]

# Compute distances for phosphate contacts
distances_phosphate = md.compute_distances(traj, contact_pairs_phosphate)

# Compute distances for uracil oxygen contacts
distances_uracil = md.compute_distances(traj, contact_pairs_uracil)

# Define the cutoff distance in nanometers
cutoff_distance = 0.5

# Filter contacts based on the cutoff distance
close_contacts_phosphate = [contact_pairs_phosphate[i] for i in range(len(contact_pairs_phosphate)) if np.any(distances_phosphate[:, i] < cutoff_distance)]
close_contacts_uracil = [contact_pairs_uracil[i] for i in range(len(contact_pairs_uracil)) if np.any(distances_uracil[:, i] < cutoff_distance)]

# Find residues that are close to RNA phosphate atoms
protein_residues_near_phosphate = set([topology.atom(pair[0]).residue for pair in close_contacts_phosphate])

# Find residues that are close to uracil oxygen atoms
protein_residues_near_uracil = set([topology.atom(pair[0]).residue for pair in close_contacts_uracil])

# Print the results
print("Amino acids in NSP13 within 0.5 nm of RNA phosphate atoms:")
for residue in sorted(protein_residues_near_phosphate, key=lambda r: r.index):
    print(f"Residue: {residue} - Index: {residue.index}")

print("\nAmino acids in NSP13 within 0.5 nm of uracil oxygen atoms:")
for residue in sorted(protein_residues_near_uracil, key=lambda r: r.index):
    print(f"Residue: {residue} - Index: {residue.index}")

Amino acids in NSP13 within 0.5 nm of RNA phosphate atoms:
Residue: ASN181 - Index: 180
Residue: CYS311 - Index: 310
Residue: SER312 - Index: 311
Residue: HIS313 - Index: 312
Residue: PRO337 - Index: 336
Residue: ALA338 - Index: 337
Residue: ARG339 - Index: 338
Residue: SER487 - Index: 486
Residue: SER488 - Index: 487
Residue: PRO516 - Index: 515
Residue: TYR517 - Index: 516
Residue: ASN518 - Index: 517
Residue: THR534 - Index: 533
Residue: ASP536 - Index: 535
Residue: THR554 - Index: 553
Residue: HIS556 - Index: 555

Amino acids in NSP13 within 0.5 nm of uracil oxygen atoms:
Residue: LYS148 - Index: 147
Residue: TYR151 - Index: 150
Residue: ASN179 - Index: 178
Residue: ASN181 - Index: 180
Residue: VAL183 - Index: 182
Residue: HIS232 - Index: 231
Residue: THR233 - Index: 232
Residue: MET235 - Index: 234
Residue: CYS311 - Index: 310
Residue: SER312 - Index: 311
Residue: HIS313 - Index: 312
Residue: PRO337 - Index: 336
Residue: ALA338 - Index: 337
Residue: ARG339 - Index: 338
Residue: AL

1. The identified residues are consistent with known RNA-binding properties and likely contribute to the RNA-helicase function of NSP13.
2. The specific pattern of interactions suggests that NSP13 has a structured RNA recognition domain, which is typical for proteins involved in RNA metabolism.
3. Understanding these interactions provides insight into drug design, especially for antiviral strategies targeting RNA-protein interfaces.